In [31]:
import os
import nltk
import re
import numpy as np
from hazm import word_tokenize, Lemmatizer, Stemmer, Normalizer
from pprint import pprint
import pandas as pd
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split

In [32]:
np.set_printoptions(suppress=True)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)

In [33]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
# InteractiveShell.ast_node_interactivity = "last_expr"

In [34]:
with open("Shams_Corpus_Paper3.txt", "r") as file:
    raw_text = file.read()

with open("persian_stopw.txt", "r") as file:
    raw_stop_words = file.read()

stop_words = word_tokenize(raw_stop_words)


def remove_persian_stopword(tokens):
    return [word for word in tokens if not word in stop_words]

In [35]:
normalizer = Normalizer()
def normalize_text(text):
    return normalizer.normalize(text)


lemmatizer = Lemmatizer()
def lemma_tokenizer(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]
#     return [lemmatizer.lemmatize(token).split("#")[0] for token in tokens]

stemmer = Stemmer()
def stem_tokenizer(tokens):
    return [stemmer.stem(token) for token in tokens]

In [36]:
def text_normalization(text):
    raw_text = re.sub(r"-+|\d+|\s+", " ", text)
    raw_text = normalize_text(raw_text)

    return raw_text


def tokenize_text(text, type="lemma"):
    tokens = word_tokenize(text)
    tokens = remove_persian_stopword(tokens)

    if type == "lemma":
        tokens = remove_persian_stopword(lemma_tokenizer(tokens))
    elif type == "stem":
        tokens = remove_persian_stopword(stem_tokenizer(tokens))

    return tokens

In [37]:
data = pd.read_csv("Shams_Corpus_Paper3.txt", names=["sentence"])
data["normalized_sent"] = data["sentence"].apply(lambda x: text_normalization(x))
data["tokens"] = data["normalized_sent"].apply(lambda x: tokenize_text(x))
data.head(10)

,sentence,normalized_sent,tokens
0,دیوان شمس تبریزی (غزلیات),دیوان شمس تبریزی (غزلیات),"[دیوان, شمس, تبریزی, غزلیات]"
1,1001 - 1500,,[]
2,--------------------------------------------------------,,[]
3,1001,,[]
4,آه در آن شمع منور چه بود\tکآتش زد در دل و دل را ربود,آه در آن شمع منور چه بود کآتش زد در دل و دل را ربود,"[آه, شمع, منور, کآتش, زد#زن, دل, دل, ربود]"
5,ای زده اندر دل من آتشی\tسوختم ای دوست بیا زود زود,ای زده اندر دل من آتشی سوختم ای دوست بیا زود زود,"[زده, دل, آتش, سوخت#سوز, دوست, زود, زود]"
6,صورت دل صورت مخلوق نیست\tکز رخ دل حسن خدا رو نمود,صورت دل صورت مخلوق نیست کز رخ دل حسن خدا رو نمود,"[دل, مخلوق, رخ, دل, حسن, خدا, رو, نمود]"
7,جز شکرش نیست مرا چاره ای\tجز لب او نیست مرا هیچ سود,جز شکرش نیست مرا چاره‌ای جز لب او نیست مرا هیچ سود,"[شکر, چاره, لب, سود]"
8,یاد کن آن را که یکی صبحدم\tاین دلم از زلف تو بندی گشود,یاد کن آن را که یکی صبحدم این دلم از زلف تو بندی گشود,"[یاد, صبحدم, دل, زلف, بست#بند, گشود#گشا]"
9,جان من اول که بدیدم تو را\tجان من از جان تو چیزی شنود,جان من اول که بدیدم تو را جان من از جان تو چیزی شنود,"[جان, بدیدم, جان, جان, شنود]"


In [38]:
tokens = data["tokens"].explode().dropna().tolist()
len(tokens)

45239

In [39]:
def concat(*iterables):
    for iterable in iterables:
        yield from iterable

In [40]:
def generate_training_data(tokens, word_to_id, window):
    X = []
    y = []
    n_tokens = len(tokens)
    unique_tokens = len(word_to_id)
    for i in range(n_tokens):
        idx = concat(
            range(max(0, i - window), i), range(i, min(n_tokens, i + window + 1))
        )
        for j in idx:
            if i == j:
                continue
            X.append(word_to_id[tokens[i]] - 1)
            y.append(word_to_id[tokens[j]] - 1)

    return np.asarray(X), np.asarray(y)

In [41]:
def generate_data(series, word_to_id, window):
    X = []
    y = []
    unique_tokens = len(word_to_id)
    for index, tokens in series.items():
        n_tokens = len(tokens)
        for i in range(n_tokens):
            idx = concat(
                range(max(0, i - window), i),
                range(i, min(n_tokens, i + window + 1))
            )
            for j in idx:
                if i == j:
                    continue
                X.append(word_to_id[tokens[i]] - 1)
                y.append(word_to_id[tokens[j]] - 1)

    return np.asarray(X), np.asarray(y)

In [42]:
window_size = 4

t = Tokenizer(filters="")
t.fit_on_texts(tokens)

sorted_count_list = sorted(t.word_counts.items(), key=lambda x: x[1], reverse=True)
word_to_id, id_to_word = t.word_index, t.index_word

X, y = generate_training_data(tokens, word_to_id, window_size)
X_sen, y_sen = generate_data(data['tokens'], word_to_id, window_size)

In [43]:
for i in sorted_count_list:
    if '#' in i[0]:
        print(i)
# sorted_count_list[:40]

('دانست#دان', 237)
('گشت#گرد', 179)
('زد#زن', 147)
('کشید#کش', 117)
('خورد#خور', 114)
('گرفت#گیر', 96)
('رسید#رس', 92)
('شکست#شکن', 67)
('برد#بر', 65)
('یافت#یاب', 65)
('نگریست#نگر', 64)
('بست#بند', 60)
('شنید#شنو', 59)
('جست#جو', 56)
('گشود#گشا', 36)
('نشست#نشین', 35)
('پرید#پر', 35)
('نمود#نما', 33)
('خواند#خوان', 32)
('گذشت#گذر', 31)
('دوید#دو', 30)
('سوخت#سوز', 28)
('افتاد#افت', 27)
('گذاشت#گذار', 26)
('مرد#میر', 25)
('ساخت#ساز', 25)
('ریخت#ریز', 24)
('ترسید#ترس', 24)
('خندید#خند', 20)
('خواست#خواه', 19)
('خرید#خر', 18)
('کاشت#کار', 18)
('آورد#آور', 17)
('پرسید#پرس', 17)
('نوشید#نوش', 17)
('آموخت#آموز', 17)
('پوشید#پوش', 15)
('شناخت#شناس', 14)
('بخشید#بخش', 13)
('پخت#پز', 12)
('خوابید#خواب', 11)
('پیچید#پیچ', 10)
('شمرد#شمار', 9)
('توانست#توان', 9)
('پذیرفت#پذیر', 9)
('باخت#باز', 9)
('رساند#رسان', 8)
('سپرد#سپر', 7)
('فرستاد#فرست', 7)
('سپارد#سپار', 7)
('فروخت#فروش', 7)
('طلبید#طلب', 7)
('شکافت#شکاف', 7)
('لرزید#لرز', 6)
('چشید#چش', 5)
('دوخت#دوز', 5)
('کشت#کش', 5)
('کوشید#کوش', 5)

In [44]:
X.shape
X_sen.shape

(361892,)

(257084,)

In [45]:
# echo 1 > /proc/sys/vm/overcommit_memory

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(X_sen)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
one_hotter = onehot_encoder.fit(integer_encoded)
X_onehot_encoded = one_hotter.transform(integer_encoded)

integer_encoded = label_encoder.fit_transform(y_sen)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y_onehot_encoded = one_hotter.transform(integer_encoded)

In [46]:
import pickle
with open("one_hot_encoder.pkl", "wb") as f: 
    pickle.dump(one_hotter, f)

In [47]:
y_onehot_encoded.shape

(257084, 8500)

In [48]:
X_onehot_encoded.shape

(257084, 8500)

In [49]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=50
)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(289513,)
(289513,)
(72379,)
(72379,)


In [50]:
X

array([1405, 1405, 1405, ..., 2649, 2649, 2649])

In [33]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Reshape, Embedding

embed_size = 50
vocab_size = X_onehot_encoded.shape[1]

model = Sequential()
model.add(
    Embedding(
        input_dim=vocab_size,
        output_dim=embed_size,
        embeddings_initializer="glorot_uniform",
        input_length=1,
    )
)
model.add(Reshape((embed_size,)))
model.add(Dense(vocab_size, kernel_initializer="glorot_uniform", activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=['mae', 'acc'])


print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 50)             418400    
                                                                 
 reshape (Reshape)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 8368)              426768    
                                                                 
Total params: 845,168
Trainable params: 845,168
Non-trainable params: 0
_________________________________________________________________
None


2022-05-15 01:08:45.964470: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-15 01:08:45.964497: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Emoji-U): /proc/driver/nvidia/version does not exist
2022-05-15 01:08:45.964714: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [34]:
X.shape

(182450,)

In [35]:
models = model.fit(
    X, y_onehot_encoded, epochs=20, validation_split=0.2, batch_size=128, verbose=1
)

2022-05-15 01:08:51.371346: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4885573120 exceeds 10% of free system memory.


Epoch 1/20
1141/1141 [==============================] - 23s 20ms/step - loss: 8.2539 - mae: 2.3881e-04 - acc: 0.0162 - val_loss: 8.0226 - val_mae: 2.3866e-04 - val_acc: 0.0147
Epoch 2/20
1141/1141 [==============================] - 22s 19ms/step - loss: 7.7416 - mae: 2.3860e-04 - acc: 0.0166 - val_loss: 8.0091 - val_mae: 2.3861e-04 - val_acc: 0.0153
Epoch 3/20
1141/1141 [==============================] - 22s 19ms/step - loss: 7.6830 - mae: 2.3854e-04 - acc: 0.0174 - val_loss: 8.0057 - val_mae: 2.3859e-04 - val_acc: 0.0159
Epoch 4/20
1141/1141 [==============================] - 22s 19ms/step - loss: 7.6581 - mae: 2.3850e-04 - acc: 0.0189 - val_loss: 8.0157 - val_mae: 2.3857e-04 - val_acc: 0.0164
Epoch 5/20
1141/1141 [==============================] - 22s 19ms/step - loss: 7.6329 - mae: 2.3845e-04 - acc: 0.0197 - val_loss: 8.0412 - val_mae: 2.3855e-04 - val_acc: 0.0165
Epoch 6/20
1141/1141 [==============================] - 23s 20ms/step - loss: 7.6049 - mae: 2.3841e-04 - acc: 0.0215 - v

In [36]:
def n_similar(word, model_, n=10):
    word_id = word_to_id[word]
    result = model_.predict([word_id]).squeeze()
    for word in (id_to_word[id + 1] for id in np.argsort(result)[::-1][0:n]):
        print(word)


In [40]:
n_similar('باغ', model_ = model, n=10)

باد
سر
جان
خاک
دل
کش
دم
عشق
لطف
خوش


In [35]:
# tokens_ = lammatized_tokens[:2000]
# t = Tokenizer(filters='')
# t.fit_on_texts(tokens_)
# sorted_count_list = sorted(t.word_counts.items(), key=lambda x: x[1], reverse=True)
# word_to_id, id_to_word = t.word_index, t.index_word
# len(word_to_id)
# X, y = generate_training_data(tokens_, word_to_id, 3)


In [36]:
# X

In [37]:
# y

In [38]:
# X.shape

In [39]:
# y.shape

In [40]:
def init_network(vocab_size, n_embedding):
    model = {
        "w1": np.random.uniform(-1, 1, (vocab_size, n_embedding)),
        "w2": np.random.uniform(-1, 1, (n_embedding, vocab_size))
    }
    return model

In [41]:
model = init_network(len(word_to_id), 10)
model["w1"].shape

(8368, 10)

In [52]:
model["w2"].shape

(10, 1171)

In [53]:
def softmax(X):
#     e_x = np.exp(x - np.max(x))
#     return e_x / e_x.sum(axis=0)

    res = []
    for x in X:
        exp = np.exp(x)
        res.append(exp / exp.sum())
    return res

In [54]:
def stable_sigmoid(x):
    sig = np.where(x < 0, np.exp(x) / (1 + np.exp(x)), 1 / (1 + np.exp(-x)))
    return sig

In [55]:
X.shape

(11988, 1171)

In [56]:
(X @ model["w1"]).shape

(11988, 10)

In [57]:
(X @ model["w1"] @ model["w2"]).shape


(11988, 1171)

In [58]:
def forward(model, X, return_cache=True):
    cache = {}
    
    cache["a1"] = X @ model["w1"]
    cache["a2"] = cache["a1"] @ model["w2"]
    print(f"a2 = {cache['a2']}")
    cache["z"] = softmax(cache["a2"])
#     cache["z"] = stable_sigmoid(cache["a2"])
    
    if not return_cache:
        return cache["z"]
    return cache

In [59]:
def cross_entropy(z, y):
    return - np.sum(np.log(z) * y)

In [66]:
def backward(model, X, y, alpha):
    cache  = forward(model, X)
#     dl_weight_inp_hidden = np.outer(target_word_vector, np.dot(weight_hidden_output, total_error.T))
#     dl_weight_hidden_output = np.outer(hidden_layer, total_error)
    da2 = cache["z"] - y
    dw2 = cache["a1"].T @ da2
    da1 = da2 @ model["w2"].T
    dw1 = X.T @ da1
    assert(dw2.shape == model["w2"].shape)
    assert(dw1.shape == model["w1"].shape)
    model["w1"] -= alpha * dw1
    model["w2"] -= alpha * dw2
    
    return cross_entropy(cache["z"], y)

In [65]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use("seaborn")

model = init_network(len(word_to_id), 10)

n_iter = 100

learning_rate = 0.01

history = [backward(model, X, y, learning_rate) for _ in range(n_iter)]

plt.clf()
plt.plot(range(len(history)), history, color="skyblue")
plt.show()

a2 = [[-2.74132539 -1.27900225 -1.80771619 ...  0.27653819  2.18316733
   0.356036  ]
 [-2.74132539 -1.27900225 -1.80771619 ...  0.27653819  2.18316733
   0.356036  ]
 [-2.74132539 -1.27900225 -1.80771619 ...  0.27653819  2.18316733
   0.356036  ]
 ...
 [-0.86869516 -1.01409124 -1.90894888 ... -0.25463134  1.92658359
   0.3085458 ]
 [-0.86869516 -1.01409124 -1.90894888 ... -0.25463134  1.92658359
   0.3085458 ]
 [-0.86869516 -1.01409124 -1.90894888 ... -0.25463134  1.92658359
   0.3085458 ]]
a2 = [[-2.30536967 -1.2409838  -1.34889173 ...  0.30970255  2.12515847
   0.32588619]
 [-2.30536967 -1.2409838  -1.34889173 ...  0.30970255  2.12515847
   0.32588619]
 [-2.30536967 -1.2409838  -1.34889173 ...  0.30970255  2.12515847
   0.32588619]
 ...
 [-0.67576492 -0.99458788 -1.69838724 ... -0.19292213  1.91841591
   0.33170993]
 [-0.67576492 -0.99458788 -1.69838724 ... -0.19292213  1.91841591
   0.33170993]
 [-0.67576492 -0.99458788 -1.69838724 ... -0.19292213  1.91841591
   0.33170993]]
a2 = [

a2 = [[-5.9980348  -5.95889827  1.36587994 ...  0.87812623  1.92214075
   0.24219688]
 [-5.9980348  -5.95889827  1.36587994 ...  0.87812623  1.92214075
   0.24219688]
 [-5.9980348  -5.95889827  1.36587994 ...  0.87812623  1.92214075
   0.24219688]
 ...
 [-4.7517591  -4.49144355  0.38128546 ...  0.76334177  2.37185902
   0.92954448]
 [-4.7517591  -4.49144355  0.38128546 ...  0.76334177  2.37185902
   0.92954448]
 [-4.7517591  -4.49144355  0.38128546 ...  0.76334177  2.37185902
   0.92954448]]
a2 = [[-1.66831947 -4.46820011  1.49254524 ...  0.92144401  1.93494083
   0.25116315]
 [-1.66831947 -4.46820011  1.49254524 ...  0.92144401  1.93494083
   0.25116315]
 [-1.66831947 -4.46820011  1.49254524 ...  0.92144401  1.93494083
   0.25116315]
 ...
 [-1.09994208 -3.26007664  0.62125857 ...  0.83668957  2.4304383
   0.97857969]
 [-1.09994208 -3.26007664  0.62125857 ...  0.83668957  2.4304383
   0.97857969]
 [-1.09994208 -3.26007664  0.62125857 ...  0.83668957  2.4304383
   0.97857969]]
a2 = [[-2

a2 = [[-3.67260638 -1.53700187  3.4852808  ...  1.72973698  2.18606308
   0.41579631]
 [-3.67260638 -1.53700187  3.4852808  ...  1.72973698  2.18606308
   0.41579631]
 [-3.67260638 -1.53700187  3.4852808  ...  1.72973698  2.18606308
   0.41579631]
 ...
 [-4.40268915 -0.23729693  5.79760099 ...  2.60515046  3.78679666
   1.94762952]
 [-4.40268915 -0.23729693  5.79760099 ...  2.60515046  3.78679666
   1.94762952]
 [-4.40268915 -0.23729693  5.79760099 ...  2.60515046  3.78679666
   1.94762952]]
a2 = [[-2.3767334  -3.04214528  3.54412939 ...  1.78290792  2.20560842
   0.42966066]
 [-2.3767334  -3.04214528  3.54412939 ...  1.78290792  2.20560842
   0.42966066]
 [-2.3767334  -3.04214528  3.54412939 ...  1.78290792  2.20560842
   0.42966066]
 ...
 [-3.27130698 -1.42870191  6.01174377 ...  2.75661738  3.89666166
   2.02379091]
 [-3.27130698 -1.42870191  6.01174377 ...  2.75661738  3.89666166
   2.02379091]
 [-3.27130698 -1.42870191  6.01174377 ...  2.75661738  3.89666166
   2.02379091]]
a2 = [

/tmp/ipykernel_17114/2941007396.py:7: RuntimeWarning: overflow encountered in exp
  exp = np.exp(x)
/tmp/ipykernel_17114/2941007396.py:8: RuntimeWarning: invalid value encountered in true_divide
  res.append(exp / exp.sum())
/tmp/ipykernel_17114/677205509.py:2: RuntimeWarning: divide by zero encountered in log
  return - np.sum(np.log(z) * y)
/tmp/ipykernel_17114/677205509.py:2: RuntimeWarning: invalid value encountered in multiply
  return - np.sum(np.log(z) * y)


a2 = [[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
a2 = [[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
a2 = [[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
a2 = [[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
a2 = [[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
a2 = [[nan nan 

KeyboardInterrupt: 

In [375]:
history

[41174.36920437467,
 34001.97985689214,
 36940.93215854208,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [341]:
model

{'w1': array([[ 2.13567115,  1.56644306, -3.31168808, ..., -1.27881225,
         -3.665346  , -0.71643332],
        [ 2.94476094,  6.58798465, -1.4649149 , ..., -2.23153483,
         -0.37211619, -2.20278355],
        [ 0.55475392,  1.25378585,  0.35800644, ..., -1.14515458,
         -1.38277165,  0.17231827],
        ...,
        [-0.3725349 , -0.38839347, -0.72622149, ..., -0.41574326,
          0.22873693,  0.41594231],
        [ 0.65206881,  0.13455332, -0.77005649, ..., -0.54984952,
          0.67393012,  0.74957822],
        [-0.06613055,  1.20413236,  0.14463711, ..., -0.24066138,
          1.16531767,  0.13553259]]),
 'w2': array([[-0.38738713, -0.20646114, -1.97325222, ..., -1.9887021 ,
          0.37277566, -0.68325111],
        [ 1.07978176,  1.52915627,  1.04595509, ...,  1.44282791,
          0.73902303, -0.02384995],
        [ 1.66933701, -1.79943934,  0.94013394, ..., -0.23462402,
          1.0196433 ,  1.28843454],
        ...,
        [-1.11376898,  0.75841651,  1.0733

In [342]:
learning = one_hot_encode(word_to_id["گرم"], len(word_to_id))
result = forward(model, [learning], return_cache=False)[0]
result

array([5.05350373e-05, 2.07845523e-03, 1.52483075e-07, ...,
       1.66334004e-04, 9.25094728e-07, 4.76827251e-06])

In [343]:
np.argsort(result)[::-1][0:5]

array([  18,   90,  533, 1048, 1127])

In [344]:
for word in (id_to_word[id] for id in np.argsort(result)[::-1][0:10]):
    print(word)


آتش
ربود
فشرد
نگار
برق
درده
وان
فرید
خبر
عدم


In [345]:
def get_word_similarities(word, model, n_similars=10):
    try:
        learning = one_hot_encode(word_to_id[word]-1, len(word_to_id))
    except KeyError:
        print(f"Word = {word} is not in corpus")
        exit()
    result = forward(model, [learning], return_cache=False)[0]
    for word in (id_to_word[id+1] for id in np.argsort(result)[::-1][0:n_similars]):
        print(word)

    

In [346]:
def get_embedding(model, word):
    try:
        idx = word_to_id[word] -1
    except KeyError:
        print("`word` not in corpus")
    one_hot = one_hot_encode(idx, len(word_to_id))
    return forward(model, one_hot)["a1"]

In [347]:
get_embedding(model, "دیو")


array([ 0.003464  , -0.59311171,  0.6253052 , -1.32282204, -0.69900121,
        2.49275488, -1.19139365, -1.16855214,  0.6965306 , -0.49444951])

In [348]:
get_word_similarities('عیش', model, 10)

خور
غلام
عشق
رقاد
قوم
عاشق
باللقاء
منت
پارس
جوع


In [349]:
get_word_similarities('میخانه', model, 10)

معاد
نسیم
مستفید
الکتب
شیفته
بلا
کرده
حمرا
خواهی_چو
اجل


In [350]:
get_word_similarities('بشر', model, 10)

جان
قوم
خور
باده
چشم
زود
خواب
دل
الفناء
نظر


In [351]:
get_word_similarities('ویرانه', model, 10)

رو
عاطفه
عاشق
سقف
لست
قافله
درآ
گنبد
انفک
شبت


In [352]:
get_word_similarities('حلال', model, 10)

گنبد
گرد
آتش
بموتی
جمیلا
نسیم
دین
الکتب
رو
اوباش


In [61]:
np.random.randn(2, 4)

array([[-0.87561612, -1.14905166,  0.10131987, -1.86758747],
       [-1.55412121,  0.31428543,  1.61102762, -1.25036518]])

In [63]:
np.random.uniform(-1, 1, (vocab_size, word_embedding_dimension))

array([[ 0.6661623 ,  0.70206294, -0.97759554,  0.75347279],
       [-0.7818824 , -0.63595427, -0.05453096,  0.15557554]])

In [33]:
from tensorflow import keras

from keras.models import Sequential 
from keras.layers import Activation, Dense, Reshape 

 
model = Sequential() 
layer_1 = Dense(16, input_shape = (8,), use_bias=False) 
model.add(layer_1) 
# layer_2 = Reshape((16,8)) 
# model.add(layer_2) 
layer_3 = Dense(4, use_bias=False) 
model.add(layer_3) 

# print(layer_2.input_shape) 
#(None, 8, 16) 
# print(layer_2.ouput_shape) 
#(None, 16, 8)

print(model.summary())
visualizer(model, format='png', view=True)


Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_44 (Dense)            (None, 16)                128       
                                                                 
 dense_45 (Dense)            (None, 4)                 64        
                                                                 
Total params: 192
Trainable params: 192
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
from tensorflow import keras
from keras.models import Sequential 
from keras.layers import Activation, Dense, Reshape 


